In [64]:
import pandas as pd
import numpy as np
import yfinance as yf
import math
from scipy import stats

In [65]:
tickers = pd.read_csv('sp500.csv', encoding='utf-8', encoding_errors='ignore',
                      header=None, usecols=[0,1], sep='\t', names =['Ticker', 'Company'])
tickers.head()

,Ticker,Company
0,A,Agilent Technologies Inc.
1,AAL,American Airlines Group Inc.
2,AAP,Advance Auto Parts Inc.
3,AAPL,Apple Inc.
4,ABBV,AbbVie Inc.


# Weighted scoring model for finding the final score

In [66]:
def createDivDf(tickers):
    col = [
        "Ticker",
        "Dividend Yeild(%)",
        "Dividend Rate",
        "Payout Ratio(%)",
        "5 Year Average Dividend Yeild(%)",
        "Earnings Growth(%)"
    ]

    div_df = pd.DataFrame(columns=col)
    for stock in tickers:
        ticker = yf.Ticker(stock)
        info = ticker.info

        div_yield = info.get("dividendYield", np.nan) * 100 if info.get("dividendYield") else np.nan
        div_rate = info.get("dividendRate", np.nan)
        payout_ratio = info.get("payoutRatio", np.nan) * 100
        five_yr_avg_div_yield = info.get("fiveYearAvgDividendYield", np.nan) * 100 if info.get("fiveYearAvgDividendYield") else np.nan
        earnings_growth = info.get("earningsGrowth", np.nan) * 100 if info.get("earningsGrowth") else np.nan

        div_df.loc[len(div_df)] = [stock, div_yield, div_rate, payout_ratio, five_yr_avg_div_yield, earnings_growth]

        # created dataframe with dividend related information

    numerical_cols = [
            "Dividend Yeild(%)",
            "Dividend Rate",
            "Payout Ratio(%)",
            "5 Year Average Dividend Yeild(%)",
            "Earnings Growth(%)"
        ]

    weights = {
            "Dividend Yeild(%) Normalized" : 0.4,
            "Dividend Rate Normalized": 0.2,
            "Payout Ratio(%) Normalized": 0.2,
            "5 Year Average Dividend Yeild(%) Normalized": 0.2,
            "Earnings Growth(%) Normalized": 0.2
        }
    for col in numerical_cols:
        normalized_col = col + " Normalized"

        if div_df[col].dropna().nunique() <= 1:
            div_df[normalized_col] = 0.5  # Neutral value if all data is NaN or same
            continue

        col_min = div_df[col].min(skipna=True)
        col_max = div_df[col].max(skipna=True)

        if col == "Payout Ratio(%)":
            div_df[normalized_col] = 1 - ((col_max - div_df[col]) / (col_max - col_min))
        else:
            div_df[normalized_col] = (div_df[col] - col_min) / (col_max - col_min)
    return div_df


In [67]:
tickers_list = tickers["Ticker"].values.tolist()
div_df = createDivDf(tickers_list) 
div_df

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: ANSS"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: HES"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: HES"}}}


,Ticker,Dividend Yeild(%),Dividend Rate,Payout Ratio(%),5 Year Average Dividend Yeild(%),Earnings Growth(%),Dividend Yeild(%) Normalized,Dividend Rate Normalized,Payout Ratio(%) Normalized,5 Year Average Dividend Yeild(%) Normalized,Earnings Growth(%) Normalized
0,A,72.0,0.99,22.900000,66.0,21.6,0.057725,0.023747,0.001168,0.053691,0.054581
1,AAL,NaN,NaN,0.000000,127.0,-10.0,NaN,NaN,0.000000,0.104866,0.039140
2,AAP,161.0,1.00,126.580000,279.0,-66.4,0.138439,0.024008,0.006458,0.232383,0.011581
3,AAPL,40.0,1.04,15.330000,54.0,12.1,0.028704,0.025052,0.000782,0.043624,0.049939
4,ABBV,280.0,6.56,303.810000,382.0,-32.4,0.246361,0.169102,0.015501,0.318792,0.028194
...,...,...,...,...,...,...,...,...,...,...,...
369,XYL,107.0,1.60,39.580002,113.0,16.1,0.089466,0.039666,0.002019,0.093121,0.051893
370,YUM,188.0,2.84,54.440000,180.0,3.9,0.162926,0.072025,0.002778,0.149329,0.045932
371,ZBH,95.0,0.96,23.410000,79.0,-34.7,0.078583,0.022965,0.001194,0.064597,0.027071
372,ZION,319.0,1.80,31.020000,335.0,27.3,0.281730,0.044885,0.001583,0.279362,0.057366


In [68]:

weights = {
            "Dividend Yield(%) Normalized" : 0.4,
            "Dividend Rate Normalized": 0.2,
            "Payout Ratio(%) Normalized": 0.2,
            "5 Year Average Dividend Yeild(%) Normalized": 0.2,
            "Earnings Growth(%) Normalized": 0.2
        }
# picked col name, picked the weight and * with value, then added the value
valid_cols = [col for col in weights.keys() if col in div_df.columns]
valid_weights = [weights[col] for col in valid_cols]
div_df["Div Score Normalized"] = div_df[valid_cols].mul(valid_weights).sum(axis=1)

div_df

,Ticker,Dividend Yeild(%),Dividend Rate,Payout Ratio(%),5 Year Average Dividend Yeild(%),Earnings Growth(%),Dividend Yeild(%) Normalized,Dividend Rate Normalized,Payout Ratio(%) Normalized,5 Year Average Dividend Yeild(%) Normalized,Earnings Growth(%) Normalized,Div Score Normalized
0,A,72.0,0.99,22.900000,66.0,21.6,0.057725,0.023747,0.001168,0.053691,0.054581,0.026638
1,AAL,NaN,NaN,0.000000,127.0,-10.0,NaN,NaN,0.000000,0.104866,0.039140,0.028801
2,AAP,161.0,1.00,126.580000,279.0,-66.4,0.138439,0.024008,0.006458,0.232383,0.011581,0.054886
3,AAPL,40.0,1.04,15.330000,54.0,12.1,0.028704,0.025052,0.000782,0.043624,0.049939,0.023879
4,ABBV,280.0,6.56,303.810000,382.0,-32.4,0.246361,0.169102,0.015501,0.318792,0.028194,0.106318
...,...,...,...,...,...,...,...,...,...,...,...,...
369,XYL,107.0,1.60,39.580002,113.0,16.1,0.089466,0.039666,0.002019,0.093121,0.051893,0.037340
370,YUM,188.0,2.84,54.440000,180.0,3.9,0.162926,0.072025,0.002778,0.149329,0.045932,0.054013
371,ZBH,95.0,0.96,23.410000,79.0,-34.7,0.078583,0.022965,0.001194,0.064597,0.027071,0.023165
372,ZION,319.0,1.80,31.020000,335.0,27.3,0.281730,0.044885,0.001583,0.279362,0.057366,0.076639


In [72]:
div_df = div_df.sort_values(by = "Div Score Normalized", ascending = False)
div_df.head(10)

,Ticker,Dividend Yeild(%),Dividend Rate,Payout Ratio(%),5 Year Average Dividend Yeild(%),Earnings Growth(%),Dividend Yeild(%) Normalized,Dividend Rate Normalized,Payout Ratio(%) Normalized,5 Year Average Dividend Yeild(%) Normalized,Earnings Growth(%) Normalized,Div Score Normalized
68,BXP,370.0,2.80,19600.000000,510.0,10.5,0.327983,0.070981,1.000000,0.426174,0.049157,0.309263
341,VNO,180.0,0.74,18.270001,487.0,1956.4,0.155670,0.017223,0.000932,0.406879,1.000000,0.285007
131,DLR,281.0,4.88,128.080000,345.0,1378.6,0.247268,0.125261,0.006535,0.287752,0.717664,0.227442
60,BKNG,71.0,38.40,25.580000,NaN,-38.2,0.056818,1.000000,0.001305,NaN,0.025360,0.205333
275,LUMN,NaN,NaN,0.000000,1194.0,NaN,NaN,NaN,0.000000,1.000000,NaN,0.200000
159,ETR,256.0,2.40,58.680000,364.0,854.5,0.224595,0.060543,0.002994,0.303691,0.461569,0.165759
301,MO,645.0,4.24,78.920000,781.0,-36.2,0.577382,0.108559,0.004027,0.653523,0.026338,0.158489
355,WHR,452.0,3.60,5384.620300,495.0,-70.6,0.402349,0.091858,0.274726,0.413591,0.009528,0.157941
62,BLK,180.0,20.84,49.840000,234.0,2.0,0.155670,0.541754,0.002543,0.194631,0.045004,0.156786
66,BTI,605.0,3.10,168.950000,776.0,1.6,0.541106,0.078810,0.008620,0.649329,0.044808,0.156313
